**분석개요**

    A. 가볼래터 성과 분석 리포트
        1. 매체별 유입 분석
            1) 유입
            2) 재방문
            3) 구독 전환
            4) 이상 유저
            5) 앱설치
            6) 활성
            7) 리텐션
        2. 이동 경로 분석

In [1]:
# basic
import gc
import os
import sys
import warnings
warnings.filterwarnings(action='ignore') 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#--------------------#
# handling
#--------------------#
import math
import time
import random
# import openpyxl
import importlib
import xlsxwriter
import numpy as np
import pandas as pd
from collections import Counter
from datetime import datetime, timedelta
from scipy.stats import pearsonr
import difflib

#--------------------#
# Vis
#--------------------#
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
## Vis for jupyter theme
from IPython.display import Image
from jupyterthemes import jtplot
jtplot.style(theme= 'grade3', context='notebook', ticks=True, grid=False) ## dark backgroud jupyter notebook theme
# plt.style.use(['seaborn-white']) ## dark background style ## white style : 'seaborn-white'
plt.style.use(['dark_background']) ## dark background style ## white style : 'seaborn-white'
## Vis for korean 
import matplotlib.font_manager as fm
font_location = '/usr/share/fonts/truetype/nanum/NanumSquareRoundB.ttf'
fprop = fm.FontProperties(fname=font_location)
font_name = fprop.get_name()
matplotlib.rc('font', family=font_name)

In [2]:
#--------------------#
# sphere package
#--------------------#
sys.path.append("/home/das_share/sphere_class/")
import SpherePackage
from SpherePackage import *
for pkg in [SpherePackage] :
    _ = importlib.reload(pkg)

#--------------------#
# kto package
#--------------------#
sys.path.append("../src/")
import kto_config
import kto_util
import kto_prep
from kto_config import *
from kto_prep import *
from kto_util import *

for pkg in [kto_config, kto_prep, kto_util] :
    _ = importlib.reload(pkg)

#-------------------#
# pickle
#-------------------#
sys.path.append("/home/minkyung62/analysis_report/00_custom_analysis/kto_custom/notebook_git/return_pickle.py")
import return_pickle
from return_pickle import *
for pkg in [return_pickle] :
    _ = importlib.reload(pkg)


# 1.Data Import

### sampling data

In [3]:
from datetime import datetime, timedelta

def date_range(start, end):
    start = datetime.strptime(start, "%Y%m%d")
    end = datetime.strptime(end, "%Y%m%d")
    dates = [date.strftime("%Y%m%d") for date in pd.date_range(start, periods=(end-start).days+1)]
    return dates

dates = date_range("20221001", "20221201")

In [4]:
lst_df = []
datess = [date[4:] for date in dates]
for _d in datess:
    df = pd.read_pickle('/home/minkyung62/test/data/' + f'{_d}' +'.pickle')
    lst_df.append(df)

df_app_log = pd.concat(lst_df)

In [7]:
# dates2 = date_range("20221202", "20221205")
# for _date in dates2:
#     _d = _date[4:]
#     df_= ReadFile.read_pickle(e_date=_date, s_date=_date)
#     df_ = df_.sample(int(len(df_) * 0.1))
#     df_.to_pickle('/home/minkyung62/test/data/'+f'{_d}' + '.pickle')

In [8]:
dates2 = date_range("20221202", "20221205")
lst_df = []
datess = [date[4:] for date in dates2]
for _d in datess:
    df = pd.read_pickle('/home/minkyung62/test/data/' + f'{_d}' +'.pickle')
    lst_df.append(df)

df_app_log_change = pd.concat(lst_df)

### original data

In [3]:
#------------------------------------------------#
# 1. log Data import
#------------------------------------------------#
# data import 1: 예산 변경 전 (~22.12.01)
s_date = '20221001'
df_app_log = ReadFile.read_pickle(e_date='20221201', s_date=s_date)
# df_app_log_copy = df_app_log.copy()        ## copy code

# # df_app_log_before = DataImport.set_date_range(df = df_app_log, start= datetime(2022,10,1), end = advertisement_date)
# # df_app_log_after = DataImport.set_date_range(df = df_app_log, start= advertisement_date, end = datetime(2022,11,30))

# # data import 2: 예산 변경 후
# CheckDate.check_date_return_pickle(e_date='20221207', s_date='20221202')
df_app_log_change = ReadFile.read_pickle(e_date='20221207', s_date='20221202')

# 2.Preprocess

In [4]:
#------------------------------------------------#
# 1. prep
#------------------------------------------------#
## 1) basic prep
df_app_log = SpherePrep.BasicPrep.basic_prep(df_app_log)   
df_app_log_change = SpherePrep.BasicPrep.basic_prep(df_app_log_change)   

## 2) user prop
df_prop = SpherePrep.Prop.df_prop_pipe(df_app_log, KEY_ID = [KEY_ID_DEVICE, KEY_ID_USER], drop_none_prop=False)       # user data
df_prop_change = SpherePrep.Prop.df_prop_pipe(df_app_log_change, KEY_ID = [KEY_ID_DEVICE, KEY_ID_USER], drop_none_prop=False)       # user data

In [5]:
## 3) param - 1
_, df_param_all = DataImport.json_to_dataframe_nodeN(
    df_app_log, [KEY_ID_DEVICE, KEY_ID_USER]
    )

_, df_param_all_ch = DataImport.json_to_dataframe_nodeN(
    df_app_log_change, [KEY_ID_DEVICE, KEY_ID_USER]
    )

## 4) referr
lst_extrated_referr_cols = ['utm_campaign', 'utm_source', 'utm_medium']

df_app_log = SpherePrep.Attribution_Prep.gen_session_referr_cols(
        df_app_log, 
        referr_col = 'referrer', 
        lst_extrated_referr_cols = lst_extrated_referr_cols,
        fill_na_organic = True)

df_app_log_change = SpherePrep.Attribution_Prep.gen_session_referr_cols(
        df_app_log_change, 
        referr_col = 'referrer', 
        lst_extrated_referr_cols = lst_extrated_referr_cols,
        fill_na_organic = True)

In [6]:
#------------------------------------------------#
# 2. param detail data
#------------------------------------------------#
## 1) event 기준
df_param_subscribe = DataImport.extract_event_param(df_param_all, EVENT_SUBSCRIBE).drop_duplicates()
df_param_subscribe_ch = DataImport.extract_event_param(df_param_all_ch, EVENT_SUBSCRIBE).drop_duplicates()

## 2) 가볼래터 data 생성
# df_param_contents_letter =\
     # df_param_all[df_param_all[PARAM_CONTENT_NAME].str.contains('가볼래-터', na=False)]

In [6]:
df_param_contents_letter[PARAM_CONTENT_NAME].unique()

array(['가볼래-터 구독하러 가기', '매월 당신의 눈앞에 여행을 배송!여행구독 <가볼래-터> 구독하러가기!',
       "여행구독 '가볼래-터' 구독자 레벨업을 위한 ★미션 참여 방법& 선물 언박싱★",
       '핫플레이스와 신상 여행정보가 가득 담긴＇가볼래-터＇구독 신청하고 매월 초간단 미션을 완료하면 특별한 선물이 담긴 여행복권을 드립니다!',
       '매월 당신의 눈앞에 여행을 배송!가볼래-터 구독하러가기!'], dtype=object)

## 1)Utils

In [7]:
# cnt -> percent function
def cnt_to_percent_ver1(df, col):
    df[col] = df[col].astype(float)

    if  df.index.dtype != 'str':
        df.index = df.index.astype('str')
    
    _lst = []
    for i in range(0,len(df)):
        x = round((df[col][i] / df[col].sum())*100, 2)
        _lst.append(x)

    df['percent']= _lst
    return df


def get_daily_cnt(dict_output, lst, nm, dict_input, df_target = df_app_log, KEY_ID = KEY_ID_DEVICE, 
                  make_sub_dict = False):
    # 1. 전체 일별 수치
    _df_target_daily = df_target[df_target[KEY_ID].isin(lst)].groupby('day')[KEY_ID]\
                    .nunique().to_frame(name = f'total_{nm}')

    # 2. 매체별 일별 수지
    for key, values in dict_input.items():
        _lst_id = list(set(lst) &  set(values))
        _df = df_target[df_target[KEY_ID].isin(_lst_id)].groupby('day')[KEY_ID]\
                    .nunique().to_frame(name = f'{key}_{nm}').fillna(0)
        if make_sub_dict == True:
            dict_output[f'{key}'] = {}

        dict_output[f'{key}'][f'{nm}'] = _df
        # dict_output[f'{key}_{nm}'] 
        _df_target_daily[f'{key}'] = _df[f'{key}_{nm}']
        _df_target_daily.fillna(0, inplace=True)
        # pd.concat([_df_target_daily, dict_output[f'{key}'][f'{key}_{nm}']], axis=1).fillna(0)
        
    return _df_target_daily

# 3.Analysis
## 0)Segment Setting
### (1)예산 변경 전

In [8]:
#------------------------------------------------#
# 1. segment by referrer
#------------------------------------------------#
## before change
dict_lst_user_from_referrs = KtoPrep.get_user_list_by_referr(
    df_app_log, KEY_ID = KEY_ID_DEVICE,
    DEPTH_1_COL = 'referrer_utm_campaign', 
    DEPTH_2_COL = 'referrer_utm_source') 
dict_lst_user_from_referrs_gudok = dict_lst_user_from_referrs['2022_gudok']
dict_lst_user_from_referrs_gudok['organic']  = dict_lst_user_from_referrs['organic']['organic'] # organic 유입 추가

#------------------------------------------------#
# 2. segment by duration per device
#------------------------------------------------#
df_duration_referrs_per_device = df_app_log.groupby([KEY_ID_DEVICE])['duration'].sum()
dict_duration_seg = dict()
dict_duration_seg['high'] = df_duration_referrs_per_device[
    df_duration_referrs_per_device >= df_duration_referrs_per_device.quantile(0.66)].index
dict_duration_seg['middle'] = df_duration_referrs_per_device[
    (df_duration_referrs_per_device < df_duration_referrs_per_device.quantile(0.66)) &
    (df_duration_referrs_per_device >= df_duration_referrs_per_device.quantile(0.33))
    ].index
dict_duration_seg['low'] = df_duration_referrs_per_device[
    df_duration_referrs_per_device < df_duration_referrs_per_device.quantile(0.33)].index

#------------------------------------------------#
# 3. segment by new or re
#------------------------------------------------#
df_app_log['created_date'] = pd.DatetimeIndex(df_app_log['created_date'])
# lst_new_device = list(df_app_log[df_app_log['created_date'] >= datetime.strptime(s_date, '%Y%m%d')][KEY_ID_DEVICE].unique())
# lst_re_device = list(df_app_log[df_app_log['created_date'] < datetime.strptime(s_date, '%Y%m%d')][KEY_ID_DEVICE].unique())

#------------------------------------------------#
# 4. segment by fraud or total
#------------------------------------------------#
lst_fraud_device = list(df_duration_referrs_per_device[df_duration_referrs_per_device < 5].index)
lst_total_device = list(df_app_log[KEY_ID_DEVICE].unique())

#------------------------------------------------#
# 5. subscribe segment 
#------------------------------------------------#
lst_subscribe_device = list(df_app_log[
    df_app_log['abs_events'].apply(lambda x : True if EVENT_SUBSCRIBE in x else False)
    ][KEY_ID_DEVICE].values)

### (2)예산 변경 후

In [9]:
#------------------------------------------------#
# 1. segment by referrer
#------------------------------------------------#
## after change
dict_lst_user_from_referrs2 = KtoPrep.get_user_list_by_referr(
    df_app_log_change, KEY_ID = KEY_ID_DEVICE,
    DEPTH_1_COL = 'referrer_utm_campaign', 
    DEPTH_2_COL = 'referrer_utm_source') 
dict_lst_user_from_referrs_gudok2 = dict_lst_user_from_referrs2['2022_gudok']
dict_lst_user_from_referrs_gudok2['organic']  = dict_lst_user_from_referrs2['organic']['organic'] # organic 유입 추가

#------------------------------------------------#
# 2. segment by duration per device
#------------------------------------------------#
df_duration_referrs_per_device2 = df_app_log_change.groupby([KEY_ID_DEVICE])['duration'].sum()
dict_duration_seg2 = dict()
dict_duration_seg2['high'] = df_duration_referrs_per_device2[
    df_duration_referrs_per_device2 >= df_duration_referrs_per_device2.quantile(0.66)].index
dict_duration_seg2['middle'] = df_duration_referrs_per_device2[
    (df_duration_referrs_per_device2 < df_duration_referrs_per_device2.quantile(0.66)) &
    (df_duration_referrs_per_device2 >= df_duration_referrs_per_device2.quantile(0.33))
    ].index
dict_duration_seg2['low'] = df_duration_referrs_per_device2[
    df_duration_referrs_per_device2 < df_duration_referrs_per_device2.quantile(0.33)].index

#------------------------------------------------#
# 3. segment by new or re
#------------------------------------------------#
df_app_log_change['created_date'] = pd.DatetimeIndex(df_app_log_change['created_date'])
# lst_new_device2 = list(df_app_log_change[df_app_log_change['created_date'] >= datetime.strptime(s_date, '%Y%m%d')][KEY_ID_DEVICE].unique())
# lst_re_device2 = list(df_app_log_change[df_app_log_change['created_date'] < datetime.strptime(s_date, '%Y%m%d')][KEY_ID_DEVICE].unique())

#------------------------------------------------#
# 4. segment by fraud or total
#------------------------------------------------#
lst_fraud_device2 = list(df_duration_referrs_per_device2[df_duration_referrs_per_device2 < 5].index)
lst_total_device2 = list(df_app_log_change[KEY_ID_DEVICE].unique())

#------------------------------------------------#
# 5. subscribe segment 
#------------------------------------------------#
lst_subscribe_device2 = list(df_app_log_change[
    df_app_log_change['abs_events'].apply(lambda x : True if EVENT_SUBSCRIBE in x else False)
    ][KEY_ID_DEVICE].values)

In [10]:
dict_lst_user_from_referrs_gudok.keys()
dict_lst_user_from_referrs_gudok2.keys()

dict_keys(['kakaotalk_bizmessage', 'GDN', 'kakao_mo', 'instagram', 'KAKAO_moment', 'FB_INSTA', 'NAVER_GFA', 'Facebook', 'organic'])

dict_keys(['FB_INSTA', 'KAKAO_moment', 'GDN', 'kakaotalk_bizmessage', 'NAVER_GFA', 'kakao_mo', 'kakaotalk_friends', 'organic'])

In [11]:
## prep referr 
#   FB_INSTA : instagram & FB_INSTA & Facebook
#   KAKAO_moment : kakao_mo & KAKAO_moment
#   kakaotalk_bizmessage --> 제외 (카카오 알림톡)

## 1) 예산 변경 전
dict_lst_user_from_referrs_gudok['KAKAO_moment'] =\
    list(
        set(
            dict_lst_user_from_referrs_gudok['kakao_mo'] + 
            dict_lst_user_from_referrs_gudok['KAKAO_moment'])
        )
dict_lst_user_from_referrs_gudok['FB_INSTA'] =\
    list(    
        set(
            dict_lst_user_from_referrs_gudok['instagram'] + 
            dict_lst_user_from_referrs_gudok['FB_INSTA']
            +dict_lst_user_from_referrs_gudok['Facebook']
            )
    )

del dict_lst_user_from_referrs_gudok['kakaotalk_bizmessage']
del dict_lst_user_from_referrs_gudok['kakao_mo']
del dict_lst_user_from_referrs_gudok['instagram']
del dict_lst_user_from_referrs_gudok['Facebook']
dict_lst_user_from_referrs_gudok.keys()

dict_keys(['GDN', 'KAKAO_moment', 'FB_INSTA', 'NAVER_GFA', 'organic'])

In [12]:
## prep referr 
#   KAKAO_moment : kakao_mo & KAKAO_moment
#   kakaotalk_bizmessage & kakaotalk_friends --> 제외 (kakaotalk_bizmessage: 카카오 알림톡)

## 2) 예산 변경 후
dict_lst_user_from_referrs_gudok2['KAKAO_moment'] =\
    list(
        set(
            dict_lst_user_from_referrs_gudok2['kakao_mo'] + 
            dict_lst_user_from_referrs_gudok2['KAKAO_moment'])
        )

del dict_lst_user_from_referrs_gudok2['kakao_mo']
del dict_lst_user_from_referrs_gudok2['kakaotalk_friends']
del dict_lst_user_from_referrs_gudok2['kakaotalk_bizmessage']
dict_lst_user_from_referrs_gudok2.keys()

dict_keys(['FB_INSTA', 'KAKAO_moment', 'GDN', 'NAVER_GFA', 'organic'])

In [13]:
dict_lst_user_from_referrs_gudok2.keys()
dict_lst_user_from_referrs_gudok.keys()

dict_keys(['FB_INSTA', 'KAKAO_moment', 'GDN', 'NAVER_GFA', 'organic'])

dict_keys(['GDN', 'KAKAO_moment', 'FB_INSTA', 'NAVER_GFA', 'organic'])

## 1)매체 성과 분석
### (1)유입
- 전체 유입
- 일별 유입
    - 전체 유입
    - 신규 유입
    - 기존 유입

In [14]:
#########################################
# 1. 매체별 전체 유입인원
## 전체 / 일별 총 유입인원
#########################################
def count_inflow(s_date, e_date, dict_input, KEY_ID = KEY_ID_DEVICE, nm = 'inflow', df_target = df_app_log):
    period = (datetime.strptime(e_date, '%Y%m%d') - datetime.strptime(s_date, '%Y%m%d')).days + 1
    dict_output_inflow_total= {}
    dict_output_inflow_daily= {}

    _df_target_daily = df_target.groupby('day')[KEY_ID]\
                            .nunique().to_frame(name = f'total_{nm}')

    for _referrer, _segment in dict_input.items():
        ## 1) 전체 하루 평균 유입인원
        dict_output_inflow_total[f'{_referrer}'] = {}
        dict_output_inflow_total[f'{_referrer}'][f'{nm}'] = len(set(_segment)) / period

        ## 2) 일별 총 유입인원
        _df = df_target[df_target[KEY_ID].isin(_segment)].groupby('day')[KEY_ID].nunique().to_frame(name = f'{_referrer}').fillna(0)
        
        dict_output_inflow_daily[f'{_referrer}'] = {}
        dict_output_inflow_daily[f'{_referrer}'][f'{nm}'] = _df
        _df_target_daily[f'{_referrer}'] = _df[f'{_referrer}']
        
        _df_target_daily.fillna(0, inplace=True)


    ### 1-2) 전체 총 유입인원 df화
    _df_output = pd.DataFrame.from_dict(dict_output_inflow_total, orient='index', columns=['inflow'])
    _df_output = cnt_to_percent_ver1(_df_output, 'inflow')
    _df_output = _df_output.sort_index(ascending=True)
    # _df_target_daily = _df_target_daily

    return _df_output, _df_target_daily

df_inflow_total_referrs, df_inflow_daily = count_inflow(s_date = '20221001', e_date = '20221201', dict_input = dict_lst_user_from_referrs_gudok)
df_inflow_total_referrs
pd.set_option('max_rows', 100)
df_inflow_daily

,inflow,percent
FB_INSTA,347.661290,0.59
GDN,600.870968,1.02
KAKAO_moment,363.532258,0.62
NAVER_GFA,557.612903,0.95
organic,57026.048387,96.83


,total_inflow,GDN,KAKAO_moment,FB_INSTA,NAVER_GFA,organic
day,,,,,,
2022-10-01,118100,6,10,11,36,117652
2022-10-02,104637,4,5,15,56,104257
2022-10-03,75539,8,5,10,38,75153
2022-10-04,71704,6,6,12,43,71267
2022-10-05,69261,5,8,12,19,68668
2022-10-06,59415,3,7,6,34,58959
2022-10-07,86043,10,8,6,47,85372
2022-10-08,136559,12,14,8,83,136024
2022-10-09,76858,12,6,4,51,76415


### (2)재방문
- 방문 빈도별 재방문율
- 주차별 재방문율

In [15]:
#------------------------------------------------#
## 2. 재방문율
#------------------------------------------------#
### 1) 1회 / 2회 이상 / 3회 이상 방문자

#### count revisit rate
def revisit_rate(dict_input, df_target = df_app_log):
    dict_output_revisit = dict()

    for _referrer, _segment in dict_input.items():
        dict_output_revisit[_referrer] = dict()
        _df_user_day_cnt = df_target[df_target[KEY_ID_DEVICE].isin(_segment)].groupby(KEY_ID_DEVICE)['day'].count()

        only_1_visit_rate = round(_df_user_day_cnt.value_counts()[1] / len(_segment), 4)
        re_visit_rate = round(1 - only_1_visit_rate, 4)
        re_visit_over_3_rate = round(1 - _df_user_day_cnt.value_counts()[:2].sum() / len(_segment), 4)
        
        dict_output_revisit[_referrer]['only_1_visit_rate'] = only_1_visit_rate
        dict_output_revisit[_referrer]['re_visit_rate'] = re_visit_rate
        dict_output_revisit[_referrer]['re_visit_over_3_rate'] = re_visit_over_3_rate

    _df_output = pd.DataFrame.from_dict(
        dict_output_revisit, orient = 'index'
        ).sort_index(ascending=True)
    
    return _df_output

df_revisit_referrs = revisit_rate(dict_lst_user_from_referrs_gudok)
df_revisit_referrs

,only_1_visit_rate,re_visit_rate,re_visit_over_3_rate
FB_INSTA,0.8448,0.1552,0.0405
GDN,0.8376,0.1624,0.0653
KAKAO_moment,0.9047,0.0953,0.0324
NAVER_GFA,0.8739,0.1261,0.0445
organic,0.8645,0.1355,0.0590


### (3)구독전환
- 구독전환수
- 구독전환율

In [16]:
#########################################
# 1. 구독완료
## 1) 일별 구독완료 유저 수
## 2) 구독전환율
#########################################
def count_subscribe_conversion(dict_input, df_target = df_app_log, 
                                df_param_sub = df_param_subscribe, KEY_ID = KEY_ID_DEVICE):
    dict_output_subscribe = {}
    df_conversion_referrs = pd.DataFrame(index=[
                                    'landing_cnt', 
                                    'subscribe_cnt', 
                                    'subscribe_conversion'])


    _df_log_target = df_target[
        df_target['abs_events'].apply(
            lambda x : True if EVENT_SUBSCRIBE in x else False
            )]

    df_output = _df_log_target.groupby('day')[KEY_ID].nunique().to_frame(name = 'total_cnt')

    for _referrer, _segment in dict_input.items():
        ### 2) 매체별 일별 구독완료 유저 수
        _df = _df_log_target[_df_log_target[KEY_ID].isin(_segment)].groupby('day')[KEY_ID].nunique().to_frame(name = f'{_referrer}_subscribe_cnt').fillna(0)

        dict_output_subscribe[f'{_referrer}'] = {}
        dict_output_subscribe[f'{_referrer}']['subscribe_cnt'] = _df
        df_output[f'{_referrer}'] = _df[f'{_referrer}_subscribe_cnt']
        df_output.fillna(0, inplace=True)

        ### 3) 구독전환율
        landing_cnt = len(_segment)
        _df = df_param_sub[df_param_sub[KEY_ID_DEVICE].isin(_segment)]
        subscribe_cnt = _df[KEY_ID_DEVICE].nunique()
        if landing_cnt !=0:
            dict_output_subscribe[f'{_referrer}']['subscribe_conversion'] = [landing_cnt,
                                                            subscribe_cnt, 
                                                            (subscribe_cnt/landing_cnt)]
        else:
            dict_output_subscribe[f'{_referrer}']['subscribe_conversion'] = [landing_cnt,
                                                            subscribe_cnt, 0]

        df_conversion_referrs[f'{_referrer}'] = dict_output_subscribe[f'{_referrer}']['subscribe_conversion']

    return df_output, df_conversion_referrs

df_daily_subscribe, df_conversion_referrs =count_subscribe_conversion(
                                                dict_lst_user_from_referrs_gudok)

pd.set_option('max_rows', 100)
df_daily_subscribe
df_conversion_referrs.T.sort_index(ascending=True)

,total_cnt,GDN,KAKAO_moment,FB_INSTA,NAVER_GFA,organic
day,,,,,,
2022-10-01,76,0.0,0.0,0.0,0.0,76
2022-10-02,66,0.0,0.0,1.0,0.0,66
2022-10-03,57,0.0,0.0,0.0,0.0,57
2022-10-04,67,0.0,0.0,0.0,0.0,66
2022-10-05,68,0.0,0.0,0.0,0.0,63
2022-10-06,73,0.0,0.0,0.0,0.0,72
2022-10-07,99,0.0,0.0,0.0,0.0,99
2022-10-08,46,0.0,0.0,0.0,0.0,46
2022-10-09,61,0.0,0.0,0.0,0.0,61


,landing_cnt,subscribe_cnt,subscribe_conversion
FB_INSTA,21555.0,459.0,0.021294
GDN,37254.0,16.0,0.000429
KAKAO_moment,22539.0,87.0,0.003860
NAVER_GFA,34572.0,151.0,0.004368
organic,3535615.0,3259.0,0.000922


### (4)Fraud
- 바운스율
- 정착율

In [17]:
#########################################
# 1. 바운스 유저 비율 & 정착 유저 비율
#########################################
def count_fraud(dict_input, lst_fraud):
    dict_fraud_referrs = {}
    dict_total_referrs  = {}

    for _referrer, _segment in dict_input.items():
        _lst_device= list(set(_segment) & set(lst_fraud))       ## 수정된 부분
        
        total_cnt = len(_segment)
        fraud_cnt = len(_lst_device)
        settle_cnt = total_cnt - fraud_cnt
        fraud_rate = fraud_cnt / total_cnt
        settle_rate = 1 - fraud_rate                                

        dict_total_referrs[f'{_referrer}'] = total_cnt
        dict_fraud_referrs[f'{_referrer}'] = [fraud_cnt, settle_cnt, fraud_rate, settle_rate]

    _df_output = pd.DataFrame.from_dict(
        dict_fraud_referrs, orient='index', 
        columns=['fraud_cnt', 'settle_cnt', 'fraud_rate', 'settle_rate']).sort_index(ascending=True)
    
    return _df_output, dict_fraud_referrs, dict_total_referrs

df_fraud_referrs, dict_fraud_referrs, dict_total_referrs1 = count_fraud(
                                        dict_input = dict_lst_user_from_referrs_gudok, lst_fraud= lst_fraud_device)
df_fraud_referrs
dict_total_referrs1

,fraud_cnt,settle_cnt,fraud_rate,settle_rate
FB_INSTA,17689,3866,0.820645,0.179355
GDN,29468,7786,0.791002,0.208998
KAKAO_moment,20269,2270,0.899286,0.100714
NAVER_GFA,30414,4158,0.879729,0.120271
organic,2654550,881065,0.750803,0.249197


{'GDN': 37254,
 'KAKAO_moment': 22539,
 'FB_INSTA': 21555,
 'NAVER_GFA': 34572,
 'organic': 3535615}

### (5)앱설치 유저

In [18]:
# #########################################
# 0. prep
# #########################################
EVENT_APP_INSTALL = '#appInstall'
app_key_android = 'qsJfZhE2Ft85SvZoXInoo3'
app_key_ios = 'qsJfZhE2Ft85SvZTOvxsUA'

#########################################
# 1. 앱설치 유저
#########################################
def count_app_install(dict_input, df_target = df_app_log, df_prop = df_prop):
    # _df_log_install = df_target[df_target['abs_events'].apply(lambda x : True if EVENT_APP_INSTALL in x else False)]
    dict_install = {}

    for _referrer, _segment in dict_input.items():
        ## 1) 캠페인 유저의 사용자 ID 추출
        _lst_campaign_user_id = list(set(df_prop[df_prop[KEY_ID_DEVICE].isin(_segment)][KEY_ID_USER]))
        
        ## 2) 캠페인 유저의 사용자 ID중 android & ios 앱 사용자
        dict_install[f'{_referrer}'] = {}
        _lst_campaign_user_id_install =\
            list(
                df_target[
                (df_target['app_key'].isin([app_key_android, app_key_ios])) & 
                (df_target[KEY_ID_USER].isin(_lst_campaign_user_id))
                ][KEY_ID_USER].unique()
                )
            # _df_log_install[_df_log_install[KEY_ID_USER].isin(_lst_user_id)][KEY_ID_USER]

        dict_install[f'{_referrer}'] = [
                                        len(_lst_campaign_user_id_install), 
                                        len(_lst_campaign_user_id_install) / len(_lst_campaign_user_id)]

    df_output = pd.DataFrame.from_dict(
                    dict_install, orient='index', columns= ['install_cnt', 'install_conversion']).sort_index(ascending=True)

    return df_output, dict_install

df_install_referrs, dict_install = count_app_install(dict_lst_user_from_referrs_gudok)
df_install_referrs

,install_cnt,install_conversion
FB_INSTA,148,0.161926
GDN,10,0.196078
KAKAO_moment,21,0.102439
NAVER_GFA,60,0.154242
organic,12877,0.425096


In [19]:
dict_install2 = {}

for _referrer, _segment in dict_lst_user_from_referrs_gudok2.items():
    ## 1) 캠페인 유저의 사용자 ID 추출
    lst_campaign_user_id = list(set(df_prop_change[KEY_ID_USER][df_prop_change[KEY_ID_DEVICE].isin(_segment)]))
    
    ## 2) 캠페인 유저의 사용자 ID중 android & ios 앱 사용자
    dict_install2[f'{_referrer}'] = {}
    lst_install_user_id =\
        list(
            df_app_log_change[KEY_ID_USER][
                (df_app_log_change['app_key'].isin([app_key_android, app_key_ios])) & 
                (df_app_log_change[KEY_ID_USER].isin(lst_campaign_user_id))]
            .unique()
            )
        # _df_log_install[_df_log_install[KEY_ID_USER].isin(_lst_user_id)][KEY_ID_USER]

    dict_install2[f'{_referrer}'] = [
                                    len(lst_install_user_id), len(lst_campaign_user_id),
                                    len(lst_install_user_id) / len(lst_campaign_user_id)]

df_output2 = pd.DataFrame.from_dict(
                dict_install2, orient='index', 
                columns= ['install_cnt', 'prop_cnt','install_conversion']).sort_index(ascending=True)

In [20]:
dict_install2
df_output2

{'FB_INSTA': [33, 341, 0.0967741935483871],
 'KAKAO_moment': [6, 76, 0.07894736842105263],
 'GDN': [1, 4, 0.25],
 'NAVER_GFA': [3, 27, 0.1111111111111111],
 'organic': [1413, 3512, 0.40233485193621865]}

,install_cnt,prop_cnt,install_conversion
FB_INSTA,33,341,0.096774
GDN,1,4,0.250000
KAKAO_moment,6,76,0.078947
NAVER_GFA,3,27,0.111111
organic,1413,3512,0.402335


### (6)활성
- 인당 체류 시간
- 인당 컨텐츠 조회수
- 검색 조회 빈도

In [21]:
#########################################
# 1. 인당 체류시간
#########################################
dict_duration = {}
dict_events_cnt_output = {}

def activation(dict_input, lst_total_device, lst_fraud, df_target = df_app_log, KEY_ID = KEY_ID_DEVICE):
    dict_duration = {}
    dict_events_cnt_output = {}

    lst_no_fraud_device = list(set(lst_total_device) - set(lst_fraud))
    _df_log_no_fraud = df_target[df_target[KEY_ID].isin(lst_no_fraud_device)]

    for _referrer, _segment in dict_input.items():
        ## 1) 평균 체류 시간
        _df = _df_log_no_fraud[_df_log_no_fraud[KEY_ID_DEVICE].isin(_segment)]
        dict_duration[f'{_referrer}']= _df.groupby(KEY_ID)['duration'].sum().mean()

        ## 2) 평균 컨텐츠 조회수
        _df_app_log_by_referr = df_target[
                                    df_target[KEY_ID_DEVICE].isin(_segment)]
        _df_app_log_by_referr['events_cnt'] = _df_app_log_by_referr['abs_events'].apply(lambda x : len(x))

        dict_events_cnt_output[f'{_referrer}'] = _df_app_log_by_referr.groupby(KEY_ID_DEVICE)['events_cnt'].sum().mean()

    ### 1-2) 평균 체류 시간 df화
    _df_output_duration = pd.DataFrame.from_dict(dict_duration, orient='index', columns=['average_duration']).sort_index(ascending=True)

    ### 2-2) 평균 컨텐츠 조회수 df화
    _df_output_page = pd.DataFrame.from_dict(dict_events_cnt_output, orient='index',  columns=['average_event_cnt']).sort_index(ascending=True)

    return _df_output_duration, _df_output_page

df_duration_referrs, df_event_cnt = activation(dict_lst_user_from_referrs_gudok, lst_total_device, lst_fraud_device)
df_duration_referrs
df_event_cnt

,average_duration
FB_INSTA,153.781136
GDN,445.031959
KAKAO_moment,305.029049
NAVER_GFA,389.746993
organic,329.714211


,average_event_cnt
FB_INSTA,2.139921
GDN,1.701992
KAKAO_moment,1.557877
NAVER_GFA,1.814416
organic,2.452247


## 2)예산 변경 후 매체 성과

In [22]:
# return all KPI after budget changed
## 1) inflow
df_inflow_total_referrs2, df_inflow_daily2 = count_inflow(s_date = '20221202', e_date = '20221207', 
                                                dict_input = dict_lst_user_from_referrs_gudok2, df_target=df_app_log_change)

## 2) revisit
df_revisit_referrs2 = revisit_rate(dict_lst_user_from_referrs_gudok2, df_target=df_app_log_change)

## 3) conversion
df_daily_subscribe2, df_conversion_referrs2 = count_subscribe_conversion(
                                                dict_lst_user_from_referrs_gudok2, df_target=df_app_log_change, 
                                                df_param_sub=df_param_subscribe_ch)

## 4) fraud
df_fraud_referrs2, dict_fraud_referrs2, dict_total_referrs2 = count_fraud(dict_lst_user_from_referrs_gudok2, lst_fraud=lst_fraud_device2)

## 5) install
df_install_referrs2, dict_install2 = count_app_install(
                                        dict_lst_user_from_referrs_gudok2, 
                                        df_target = df_app_log_change, df_prop=df_prop_change)

## 6) duration
df_duration_referrs2, df_event_cnt2 = activation(
                                        dict_lst_user_from_referrs_gudok2, lst_total_device = lst_total_device2, 
                                        lst_fraud = lst_fraud_device2, df_target=df_app_log_change)

In [23]:
df_inflow_daily2

,total_inflow,FB_INSTA,KAKAO_moment,GDN,NAVER_GFA,organic
day,,,,,,
2022-12-02,56699,1303,252,512,25,46356
2022-12-03,51856,1383,304,662,16,43785
2022-12-04,54788,1645,292,416,12,47361
2022-12-05,51339,1452,319,336,460,43688
2022-12-06,72893,1380,318,248,715,48621
2022-12-07,61828,1303,255,212,730,49645


In [24]:
df_daily_subscribe2

,total_cnt,FB_INSTA,KAKAO_moment,GDN,NAVER_GFA,organic
day,,,,,,
2022-12-02,86,29,5,1.0,0.0,47
2022-12-03,87,32,11,0.0,0.0,50
2022-12-04,73,33,5,0.0,0.0,36
2022-12-05,70,32,6,0.0,6.0,29
2022-12-06,81,33,10,1.0,9.0,29
2022-12-07,79,27,5,1.0,1.0,43


In [25]:
# merge all output 
lst_df1 = [df_inflow_total_referrs, df_revisit_referrs, df_conversion_referrs.T,\
            df_fraud_referrs, df_install_referrs, df_duration_referrs, df_event_cnt]
lst_df2 = [df_inflow_total_referrs2, df_revisit_referrs2, df_conversion_referrs2.T,\
            df_fraud_referrs2, df_install_referrs2, df_duration_referrs2, df_event_cnt2]

def merge_df(lst_df1, lst_df2):
    df_total_output = pd.DataFrame()
    df_total_output2 = pd.DataFrame()

    for _df1, _df2 in zip(lst_df1, lst_df2):
        df_total_output = pd.merge(df_total_output, _df1[:-1], left_index=True, right_index=True, how='outer')
        df_total_output2 = pd.merge(df_total_output2, _df2[:-1], left_index=True, right_index=True, how='outer')
    
    # _df1.drop(columns='percent', axis=1, inplace=True)
    # _df2.drop(columns='percent', axis=1, inplace=True)

    return df_total_output, df_total_output2

df_total_output, df_total_output2 = merge_df(lst_df1=lst_df1, lst_df2=lst_df2)

df_total_output
df_total_output2
## df_total_output2에서 naver의 평균 체류시간이 결측치인 이유는 lst_device_no_fraud와 naver로 들어온 기기 간의 교집합이 존재하지 않기 때문

,inflow,percent,only_1_visit_rate,re_visit_rate,re_visit_over_3_rate,landing_cnt,subscribe_cnt,subscribe_conversion,fraud_cnt,settle_cnt,fraud_rate,settle_rate,install_cnt,install_conversion,average_duration,average_event_cnt
FB_INSTA,347.661290,0.59,0.8448,0.1552,0.0405,21555.0,459.0,0.021294,17689,3866,0.820645,0.179355,148,0.161926,153.781136,2.139921
GDN,600.870968,1.02,0.8376,0.1624,0.0653,37254.0,16.0,0.000429,29468,7786,0.791002,0.208998,10,0.196078,445.031959,1.701992
KAKAO_moment,363.532258,0.62,0.9047,0.0953,0.0324,22539.0,87.0,0.003860,20269,2270,0.899286,0.100714,21,0.102439,305.029049,1.557877
NAVER_GFA,557.612903,0.95,0.8739,0.1261,0.0445,34572.0,151.0,0.004368,30414,4158,0.879729,0.120271,60,0.154242,389.746993,1.814416


,inflow,percent,only_1_visit_rate,re_visit_rate,re_visit_over_3_rate,landing_cnt,subscribe_cnt,subscribe_conversion,fraud_cnt,settle_cnt,fraud_rate,settle_rate,install_cnt,install_conversion,average_duration,average_event_cnt
FB_INSTA,1349.000000,2.90,0.9091,0.0909,0.0175,8094.0,186.0,0.022980,6558,1536,0.810230,0.189770,33,0.096774,145.643262,2.143316
GDN,344.166667,0.74,0.8600,0.1400,0.0499,2065.0,3.0,0.001453,1659,406,0.803390,0.196610,1,0.250000,380.705586,1.612591
KAKAO_moment,276.333333,0.59,0.9071,0.0929,0.0356,1658.0,42.0,0.025332,1372,286,0.827503,0.172497,6,0.078947,244.972259,2.169481
NAVER_GFA,314.166667,0.68,0.9443,0.0557,0.0143,1885.0,16.0,0.008488,1669,216,0.885411,0.114589,3,0.111111,261.726829,1.590451


In [26]:
# df_app_log_gudok_before = df_app_log[df_app_log['referrer_utm_campaign'] == '2022_gudok']
# df_app_log_gudok_after = df_app_log_change[df_app_log_change['referrer_utm_campaign'] == '2022_gudok']

# df_app_log_gudok_before_long = df_app_log_gudok_before.groupby(['day','referrer_utm_source'])[KEY_ID_DEVICE].nunique().reset_index()
# df_app_log_gudok_after_long = df_app_log_gudok_after.groupby(['day','referrer_utm_source'])[KEY_ID_DEVICE].nunique().reset_index()

# df_vis_before_wide = pd.pivot(df_app_log_gudok_before_long, index='day', columns='referrer_utm_source', values=KEY_ID_DEVICE).fillna(0)
# df_vis_after_wide = pd.pivot(df_app_log_gudok_after_long, index='day', columns='referrer_utm_source', values=KEY_ID_DEVICE).fillna(0)
# pd.concat([df_vis_before_wide, df_vis_after_wide])

In [27]:
# calculate size of all kpi changes
def size_of_changes(df_before = df_total_output, df_after = df_total_output2):
    # 1) counting size of change
    df_amount_of_change = pd.DataFrame()
    
    ## 각 KPI의 before vs after 간의 차이 집계
    for _col in [
        'inflow', 'fraud_rate', 're_visit_rate', 'subscribe_cnt', 'subscribe_conversion', 
        'install_cnt', 'install_conversion','average_duration']:
        df_amount_of_change[_col] = df_after[_col] - df_before[_col]
    
    # # 2) counting cost of diff
    # df_output_copy = pd.DataFrame(index = df_amount_of_change.index)
    # df_output_copy2 = pd.DataFrame(index = df_amount_of_change.index)
    # df_kpi_cost = pd.DataFrame(index = df_amount_of_change.index)
    # df_kpi_cost_change = pd.DataFrame(index = df_amount_of_change.index)

    # df_cost = pd.DataFrame(index = df_amount_of_change.index)
    # df_cost_rate = pd.DataFrame(index = df_amount_of_change.index)

    # df_output_copy['costs']= pd.DataFrame.from_dict(dict_cost['before'], orient='index')[0]
    # df_output_copy2['costs'] = pd.DataFrame.from_dict(dict_cost['after'], orient='index')[0]
    # df_cost['diff_cost'] = df_output_copy2['costs'] - df_output_copy['costs']

    df_change_rate = pd.DataFrame(index = df_amount_of_change.index)\

    for _kpi, in zip(['inflow', 'fraud_rate', 'subscribe_conversion','average_duration']):
        ## kpi 증감률
        df_change_rate[f'{_kpi}_change_rate'] =\
        round( (df_after[_kpi] - df_before[_kpi]) / df_before[_kpi], 3)

        # ## 비용 차이값
        # df_kpi_cost_change[f'{txt}_costs'] = round(df_output_copy2['costs']/df_after[_kpi], 2).to_frame()
        # df_kpi_cost[f'{txt}_costs']  = round(df_output_copy['costs']/df_before[_kpi], 2).to_frame()

        # df_cost[f'{txt}_costs'] =\
        #     round(df_output_copy2['costs']/df_after[_kpi] - df_output_copy['costs']/df_before[_kpi], 2)

    ## 결측치 및 inf 제거: 일단 처리 안 하기로 --> 애초에 차이가 없어서 0인 경우와 값이 없어서 결측치인 경우를 구분하기 위해
    # for _df in [df_amount_of_change, df_cost, df_cost_rate]:
    #     _df.replace([np.inf, -np.inf], np.nan, inplace = True)
    #     _df.fillna(0, inplace=True)

    df_change_rate.fillna(0, inplace=True)
    return df_amount_of_change, df_change_rate

# show output
df_diff_kpi, df_diff_rate =\
    size_of_changes(df_before = df_total_output, df_after = df_total_output2)

df_diff_kpi
df_diff_rate

,inflow,fraud_rate,re_visit_rate,subscribe_cnt,subscribe_conversion,install_cnt,install_conversion,average_duration
FB_INSTA,1001.338710,-0.010415,-0.0643,-273.0,0.001686,-115,-0.065151,-8.137873
GDN,-256.704301,0.012388,-0.0224,-13.0,0.001023,-9,0.053922,-64.326373
KAKAO_moment,-87.198925,-0.071783,-0.0024,-45.0,0.021472,-15,-0.023492,-60.056790
NAVER_GFA,-243.446237,0.005682,-0.0704,-135.0,0.004120,-57,-0.043131,-128.020165


,inflow_change_rate,fraud_rate_change_rate,subscribe_conversion_change_rate,average_duration_change_rate
FB_INSTA,2.880,-0.013,0.079,-0.053
GDN,-0.427,0.016,2.383,-0.145
KAKAO_moment,-0.240,-0.080,5.563,-0.197
NAVER_GFA,-0.437,0.006,0.943,-0.328


In [28]:
def calculate_kpi_average(index_input, dict_cost, df_daily_input1, df_daily_input2, df_target1, df_target2):
    # 1) daily average inflow costs per 1 person
    df_kpi_average_before = pd.DataFrame(index = df_daily_input1.index)
    df_kpi_average_after = pd.DataFrame(index = df_daily_input2.index)

    before_period = (datetime.strptime('20221201', '%Y%m%d') - datetime.strptime('20221001', '%Y%m%d')).days + 1
    after_period = (datetime.strptime('20221207', '%Y%m%d') - datetime.strptime('20221202', '%Y%m%d')).days + 1

    dict_average_daily_costs = {}
    dict_average_daily_costs['before'] = {}
    dict_average_daily_costs['after'] = {}

    for _k, _v in dict_cost['before'].items():
        dict_average_daily_costs['before'][f'{_k}'] = dict_cost['before'][f'{_k}'] / before_period

    for _k, _v in dict_cost['after'].items():
        dict_average_daily_costs['after'][f'{_k}'] = dict_cost['after'][f'{_k}'] / after_period

    for _col in index_input:
        df_kpi_average_before[_col] = pd.DataFrame(round(dict_average_daily_costs['before'][_col] / df_daily_input1[_col], 2))
        df_kpi_average_after[_col] = pd.DataFrame(round(dict_average_daily_costs['after'][_col] / df_daily_input2[_col], 2))
    
    # 결측치 및 inf 값 0으로 처리
    for _df in [df_kpi_average_before, df_kpi_average_after]:
        _df.replace([np.inf, -np.inf], np.nan, inplace = True)
        _df.fillna(0, inplace=True)

    _df1 = round(df_kpi_average_before.mean(), 2).to_frame(name = 'inflow_costs_before')
    _df2 = round(df_kpi_average_after.mean(), 2).to_frame(name = 'inflow_costs_after')
    df_total_costs_average = pd.concat([_df1, _df2], axis = 1)

    # 2) diff of average bounce rate between before and after
    dict_output_total_compare = {}
    # dict_output_total_compare['inflow_costs'] = {}
    # dict_output_total_compare['inflow_costs']['before'] = round(_df1['inflow_costs_before'].mean(), 2)
    # dict_output_total_compare['inflow_costs']['after'] = round(_df2['inflow_costs_after'].mean(), 2)

    for _col in ['inflow','fraud_rate', 'subscribe_conversion', 'average_duration']:
        dict_output_total_compare[_col] = {}
        dict_output_total_compare[_col]['before'] = round(df_target1[_col].mean(), 4)
        dict_output_total_compare[_col]['after'] = round(df_target2[_col].mean(), 4)

    df_final_output = pd.DataFrame.from_dict(dict_output_total_compare, orient = 'columns')
    
    return df_kpi_average_before, df_kpi_average_after, df_total_costs_average, df_final_output

lst_index = list(df_total_output.index)
dict_costs = {}
dict_costs['before'] = {'FB_INSTA': 27500000, 'GDN': 20000000, 'KAKAO_moment': 27500000, 'NAVER_GFA': 59090909}
dict_costs['after'] = {'FB_INSTA': 54410000, 'GDN': 21538000, 'KAKAO_moment': 20328000,'NAVER_GFA': 53724000}

df_average_inflow_costs_before, df_average_inflow_costs_after, df_total_costs_average, df_final_output =\
    calculate_kpi_average(
        lst_index, dict_costs, df_inflow_daily, 
        df_inflow_daily2, df_total_output, df_total_output2)

In [29]:
# df_average_inflow_costs_before
# df_average_inflow_costs_after

In [30]:
df_total_costs_average
df_final_output

,inflow_costs_before,inflow_costs_after
FB_INSTA,27863.46,6467.58
GDN,15375.58,10525.49
KAKAO_moment,22595.82,11792.16
NAVER_GFA,10846.41,284700.95


,inflow,fraud_rate,subscribe_conversion,average_duration
before,467.4194,0.8477,0.0075,323.3973
after,570.9167,0.8316,0.0146,258.2620


In [31]:
# 매체별 총 유입자 수
##  before
for _k, _r in dict_lst_user_from_referrs_gudok.items():
    _k, len(set(_r))

print('-'*70)

## after
for _k, _r in dict_lst_user_from_referrs_gudok2.items():
    _k, len(set(_r))

('GDN', 37254)

('KAKAO_moment', 22539)

('FB_INSTA', 21555)

('NAVER_GFA', 34572)

('organic', 3535615)

...

('FB_INSTA', 8094)

('KAKAO_moment', 1658)

('GDN', 2065)

('NAVER_GFA', 1885)

('organic', 265316)